In [7]:
# General Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Web Scraping Libraries
import urllib
import requests
from bs4 import BeautifulSoup

# Regex Library
import re

# Time-related Libraries
import time

# NLP Libraries
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

# Helper functions
import MVP_Bojado, MVP_Shi

# Environment file
import env, env_Shi

import warnings
warnings.filterwarnings("ignore")

## Data Acquisition

### URL Format of Indeed.com
1. Search chemist in TX<br>
https://www.indeed.com/jobs?q=chemist&l=TX
2. Search chemist in San Antonio, TX<br>
https://www.indeed.com/jobs?q=chemist&l=San+Antonio%2C+TX
3. Search data scientist in San Antonio, TX<br>
https://www.indeed.com/jobs?q=data+scientist&l=San+Antonio%2C+TX
4. Search data scientist intern in San Anotnio, TX<br>
https://www.indeed.com/jobs?q=data+scientist+intern&l=San+Antonio%2C+TX
5. Sort the data scientist jobs posting by date<br>
https://www.indeed.com/jobs?q=data+scientist&l=San+Antonio%2C+TX&sort=date

**Takeaways**
1. q = job title
2. l = location

### URL Format of Monster.com
https://www.monster.com/jobs/search/?q=data-scientist&where=San-Antonio__2C-TX

### Generate the URL of a Job Search at Indeed.com

In [2]:
def first_page_url_indeed(job_title, location):
    '''
    This function returns a URL of the 1st page of a job search at Indeed.com 
    based on the job title and the location.
    '''
    # Create the base URL for a job serch at Indeed.com
    base_url = 'https://www.indeed.com/jobs?'
    # Create a dictionary to map the keys to the input parameters
    dic = {'q': job_title, 'l': location, 'sort': 'date'}
    # Convert the dictionary to a query string
    relative_url = urllib.parse.urlencode(dic)
    # Generate the full URL of the first page
    url = base_url + relative_url
    return url

In [42]:
# Test the function
url = first_page_url_indeed('data scientist', 'al')
url

'https://www.indeed.com/jobs?q=data+scientist&l=al&sort=date'

### Make the HTTP Request

In [49]:
def first_page_soup_indeed(job_title, location):
    '''
    This function returns a BeautifulSoup object to hold the content 
    of the first page of a request for job searching at Indeed.com
    '''
    # Generate the URL of the job search based on title and location
    url = first_page_url_indeed(job_title, location)
    # Make the HTTP request
    response = requests.get(url)
    # Print the status code of the request
    print("Status code of the request: ", response.status_code)
    # Sanity check to make sure the document type is HTML
    print("Document type: ", response.text[:15])
    # Take a break
    time.sleep(5)
    # Make a soup to hold the response content
    soup = BeautifulSoup(response.content, "html.parser")
    # Print out the title of the content
    print("Title of the response: ", soup.title.string)
    return soup

In [52]:
first_page_soup = first_page_soup_indeed("data scientist", 'al')
type(first_page_soup)

Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com


bs4.BeautifulSoup

In [12]:
# Find out the tag that contains the number of the jobs by seaching

num_jobs = first_page_soup.find('div', id='searchCountPages')
print("Data Type: ", type(num_jobs))
print("Name of the Tag: ", num_jobs.name)
print("Attributes of the Tag: ", num_jobs.attrs)
print("Text within the Tag: ")
num_jobs.text

Data Type:  <class 'bs4.element.Tag'>
Name of the Tag:  div
Attributes of the Tag:  {'id': 'searchCountPages'}
Text within the Tag: 


'\n                    Page 1 of 560 jobs'

In [13]:
# Find the number of the jobs in the text
match = re.findall(r'(\d+)', num_jobs.text)
match[1]

'560'

In [9]:
def num_jobs_indeed(first_page_soup):
    '''
    This function returns the total number of the jobs in the searching result.
    '''
    # Find out the section contains total number of jobs  
    div = first_page_soup.find('div', id='searchCountPages')
    # Extract the number
    num_jobs = re.findall(r'(\d+)', div.text)[1]
    return num_jobs

In [53]:
# Test the function num_jobs_indeed
num_jobs_indeed(first_page_soup)

'40'

In [12]:
def page_num_indeed(url):
    '''
    This function returns the page number of job searching results. 
    '''
    # Create a Soup object based on the url
    soup = page_soup_indeed(url)
    # Find out the section contains total number of jobs  
    div = soup.find('div', id='searchCountPages')
    # Extract the number
    page_num = re.findall(r'(\d+)', div.text)[0]
    return page_num

In [54]:
# Test the function num_jobs_indeed
page_num_indeed(first_page_soup)

'1'

In [14]:
# Define a function to extract all job cards in a Indeed page

def job_cards_indeed(soup):
    '''
    This function accepts the Soup object of a Indeed page 
    return an iterator containing the all the job cards in this page.
    '''
    # Find the appropriate tag that contains all of the job listings in this page
    tag = soup.find('td', id="resultsCol")
    # Extract all job cards
    job_cards = tag.find_all('div', class_='jobsearch-SerpJobCard')
    return job_cards

In [55]:
# Test the function job_cards_indeed
job_cards = job_cards_indeed(first_page_soup)

# Print the data type of job_cards
type(job_cards)

bs4.element.ResultSet

**Quick Note**: job_cards is an iterator

In [56]:
# How many jobs listed in the 1st page? 
len(job_cards)

15

In [17]:
def job_titles_indeed(job_cards):
    '''
    This function extract the job titles from a job_cards set. 
    '''
    # Create a list to hold the job titles
    titles = []
    # For Loop throught the job cards to extract the titles
    for job in job_cards:
        title = job.find('h2', class_='title')
        title = title.text.strip()
        titles.append(title)
    return titles

In [57]:
titles = job_titles_indeed(job_cards)
titles

['Data Scientist\nnew',
 'MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE\nnew',
 'Data Scientist\nnew',
 'Data Analyst - Microsoft Stack (mid-senior)\nnew',
 'Statistical Analyst\nnew',
 'Data Scientist Intern\nnew',
 'Machine Learning/Artificial Intelligence Software Developer\nnew',
 'Software Engineer/Data Scientist\nnew',
 'Lead Financial Analyst - Artificial Intelligence Strategic G...\nnew',
 'BI Architect/Data Scientist',
 'Asst Research Professional - Research Data Scientist',
 '2021-18 Software Engineers for BMDS Data Analysis Suite',
 'Cyber Artificial Intelligence (AI) SME',
 '2021-02 Artificial Intelligence Designer',
 'Deep Learning Engineer']

In [19]:
# Define a function to pull the company names from a set of job cards

def company_names_indeed(job_cards):
    '''
    This function extracts the company names from a set of job cards.
    '''
    # Create a list to hold the company names
    names = []
    # For loop through the job cards to pull the company names
    for job in job_cards:
        name = job.find('span', class_='company')
        name = name.text.strip()
        names.append(name)
    return names

In [58]:
# Test the function: comany_names_indeed

company_names = company_names_indeed(job_cards)
company_names

['Quiq Inc',
 'B.A.S.S., LLC',
 'Vision',
 'Vaco',
 'The Personnel Board of Jefferson County',
 'LOCKHEED MARTIN CORPORATION',
 'IERUS Technologies, Inc.',
 'Torch Technologies, Inc.',
 'Deloitte',
 'Doozer Software',
 'The University of Alabama',
 '1st Edge',
 'Quantum Research International, Inc.',
 '1st Edge',
 'Numerator']

In [21]:
# Define a function to pull the post ages from a set of job cards

def post_ages_indeed(job_cards):
    '''
    This function pulls the post ages from a set of job cards.
    '''
    # Create a list to hold the post ages
    ages = []
    # For loop through the job cards to pull the post ages
    for job in job_cards:
        age = job.find('span', class_='date')
        age = age.text.strip()
        ages.append(age)
    return ages

In [59]:
# Test the function: post_ages_indeed
ages = post_ages_indeed(job_cards)
ages

['Today',
 'Today',
 'Today',
 'Today',
 '2 days ago',
 '4 days ago',
 '5 days ago',
 '6 days ago',
 '6 days ago',
 '11 days ago',
 '11 days ago',
 '12 days ago',
 '22 days ago',
 '21 days ago',
 '28 days ago']

In [23]:
# Define a function to pull the location from a set of job cards

def job_locations_indeed(job_cards):
    '''
    This function pulls the job locations from a set of job cards.
    '''
    # Create a list to hold the locations
    locations = []
    # For loop through the job cards to pull the locations
    for job in job_cards:
        location = job.find('div', class_='location accessible-contrast-color-location')
        if location == None:
            location = job.find('span', class_='location accessible-contrast-color-location')
        location = location.text.strip()
        locations.append(location)
    return locations

In [60]:
locations = job_locations_indeed(job_cards)
locations

['United States',
 'Birmingham, AL 35243',
 'Huntsville, AL',
 'Hartselle, AL',
 'Jefferson County, AL',
 'Huntsville, AL 35806',
 'Huntsville, AL 35805',
 'Huntsville, AL 35802',
 'Birmingham, AL 35203 (Central City area)',
 'Birmingham, AL 35216',
 'Tuscaloosa, AL',
 'Huntsville, AL',
 'Huntsville, AL 35806',
 'Huntsville, AL',
 'Alabama']

In [25]:
# Define a function to pull the company ratings from a set of job cards

def company_rating_indeed(job_cards):
    '''
    This function pulls the company rating from a set of job cards.
    If the rating is unavailable, it will be marked as 'missing'.
    '''
    # Create a list to hold the locations
    ratings = []
    # For loop through the job cards to pull the locations
    for job in job_cards:
        rating = job.find('span', class_='ratingsContent')
        if rating == None:
            ratings.append('missing')
            continue
        rating = rating.text.strip()
        ratings.append(rating)
    return ratings

In [61]:
ratings = company_rating_indeed(job_cards)
ratings

['missing',
 'missing',
 'missing',
 '3.7',
 'missing',
 '4.0',
 '4.7',
 'missing',
 '4.0',
 '4.8',
 '4.4',
 'missing',
 '4.0',
 'missing',
 '3.6']

In [27]:
def acuqire_indeed_job_description(url):
    '''
    This function accepts the URL of a job posting and pull its description.
    '''
    # Make the HTTP request
    request = requests.get(url)
    print("Status Code: ", request.status_code)
    # Take a break
    time.sleep(5)
    # Make a soup variable holding the response content
    soup = BeautifulSoup(request.content, "html.parser")
    if soup == None:
        description = 'error'
    else:
        # Print the page's title
        print(soup.title.string)
        # Find the section that contains job description
        description = soup.find('div', id="jobDescriptionText")
        if description == None:
            description = 'error'
        else:
            description = description.text
    return description

def job_links_and_contents_indeed(job_cards):
    '''
    This function pulls the job links and descriptions from a set of job cards.
    '''
    # Create a list to hold the links and descriptions
    links = []
    descriptions = []
    # For loop through the job cards to pull the links and descriptions
    for job in job_cards:
        link = job.find('a')['href']
        link = 'https://www.indeed.com' + link
        link = link.replace(';', '&')
        description = acuqire_indeed_job_description(link)
        links.append(link)
        descriptions.append(description)
    return links, descriptions

In [62]:
# Test the function: job_links_and_contents_indeed
links, descriptions = job_links_and_contents_indeed(job_cards)

Status Code:  200
Data Scientist - United States - Indeed.com
Status Code:  200
MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE - Birmingham, AL 35243 - Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL - Indeed.com
Status Code:  200
Vaco Careers and Employment | Indeed.com
Status Code:  200
Statistical Analyst - Jefferson County, AL - Indeed.com
Status Code:  200
Data Scientist Intern - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Machine Learning/Artificial Intelligence Software Developer - Huntsville, AL 35805 - Indeed.com
Status Code:  200
Software Engineer/Data Scientist - Huntsville, AL 35802 - Indeed.com
Status Code:  200
Lead Financial Analyst - Artificial Intelligence Strategic Growth Offering (AI SGO) Finance & Investment - Birmingham, AL 35203 - Indeed.com
Status Code:  200
BI Architect/Data Scientist - Birmingham, AL 35216 - Indeed.com
Status Code:  200
Asst Research Professional - Research Data Scientist - Tuscaloosa, AL - Indeed.com
Status Code:  200
2021-18 S

In [63]:
links

['https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B9JzWmK0GDxRzYzuZf9xSyXN8pQP8ihv6GH-rkAji3LyaR-hLXYB_NfHRnutLWTuqKBdImdeUix5pYB3Uv4mWnHLZlzd5py00o1lMFznkqSTkX2n8aY8mwnVhJALIJXce_3bj6Yi5-UDH-VhWVn3o6zeb2Pm8VmBf7XYjX1VtegcSkVtTpQAVHk_cSXYlbbb9ppduWw_Qf5XW46x30WeNoo3_22bLgKZLzhJEV_E3Ixv1DiqZPYzqp0IMYZ0Djwgv954h1Bqmz53jK2UmTAk6lmmvbIoTkfhAW5z9S56qUXHWlwvic9MvReVXyrEpYz_zaY4XbmDDXr3uIHFCGuQKnKzNsLz5vp9EkzV1o5kbpCj6qti0RZ7NseG1iXFMZ2GODxurbs811H733Ly8Iz-Y8CM0wqrwTs_8vPgE0g-b_Tx-QYca5xn1D&p=0&fvj=1&vjs=3',
 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DVr7NBsl4cHSE4aBurKzaRDI_6xQvAm7MDK25NP7GKwrPFBHjnIhnnMkAKtOTcAVcwvdJYQ1iX_OUlRkCcJ9lYHn6yw2Xoq1V-R4Wdf5ZtG2pRL3zKYjPaUIJ7FfKiCmO7nZu9zD3akcI0SzfFz7oRw23qh6o47s6hI0zsqL0wfiKJ4EFcwcqvgr2zn3ygOEqvyjn_BqruYjouMp_OPBKXN1sJf_xD6jAidmRKsWVQCbimyT8pNek2EIcd1SHGimJOFbqnE5QnWqRtOwd1Em2mVe_WF7eRHtG2Ad9d5Btcaz-YQLMoJ5pB8XdgfSqvtHMJyTpI5ilFsZgBFjZL1WHSJK1XUvqlXQd2hYnYwqyUbescsJP9n5A1Qu-e6G6MmRKAdUa65Lq5aeDjyXXxYGCtPmDVqGApWN3X_1kyn3niS0FsFkisS

In [64]:
descriptions[1]

"MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCEBasic Duties & Responsibilities: Query and report on dataBuild and maintain analytical tools, metrics & models across the following channels: membership, sales, marketing and tournaments.Create compelling data visualizations and stories across internal LOB’sResearch internal data in order to build out analytics for both sales and marketing benefitsOwnership of data flow management between our internal systems and 3rd party systems such as PCD, Dovetail, Salesforce, US Sweeps and morePresent information to organizational stakeholders and executivesQualifications: Bachelor’s Degree in mathematics, engineering, computer science, data science, or other relevant fieldStrong written & verbal communication as well as visualization skillsProficiency in analytical/statistical methods and technologies (e.g. SQL, R, Python, Tableau, Microsoft BI, etc)Familiarity with concepts such as predictive modelling, cloud computing, machine learning, artificial inte

In [35]:
# Define a function to create a Soup object based on a job search url

def page_soup_indeed(url):
    '''
    This function returns a BeautifulSoup object to hold the content 
    of a page for a job searching results at Indeed.com
    '''
    # Make the HTTP request
    response = requests.get(url)
    # Print the status code of the request
    print("Status code of the request: ", response.status_code)
    # Sanity check to make sure the document type is HTML
    print("Document type: ", response.text[:15])
    # Take a break
    time.sleep(5)
    # Make a soup to hold the response content
    soup = BeautifulSoup(response.content, "html.parser")
    # Print out the title of the content
    print("Title of the response: ", soup.title.string)
    return soup

In [84]:
# Test the function: page_soup_indeed

url = 'https://www.indeed.com/jobs?q=data+scientist&l=al&sort=date'
soup = page_soup_indeed(url)
type(soup)

Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com


bs4.BeautifulSoup

In [86]:
# Find out the page number
int(page_num_indeed(soup))

1

In [38]:
# Pull the job cards from the soup
type(job_cards_indeed(soup))

bs4.element.ResultSet

In [99]:
# Define a function to pull job information from a job search URL

def acquire_page_indeed(url):
    '''
    This function accepts a job search URL and returns a pandas dataframe 
    containing job title, location, company, company rating, post age and description. 
    '''
    # Create a Soup object based on the url
    soup = page_soup_indeed(url)
    # Pull the job cards
    job_cards = job_cards_indeed(soup)
    # Pull the job titles
    titles = job_titles_indeed(job_cards)   
    # Pull the names of the companies
    companies = company_names_indeed(job_cards)
    # Pull the post ages
    ages = post_ages_indeed(job_cards)
    # Pull the job locations
    locations = job_locations_indeed(job_cards)
    # Pull the company ratings
    ratings = company_rating_indeed(job_cards)
    # Pull the hyperlinks and job description
    links, descriptions = job_links_and_contents_indeed(job_cards)    
    # Create a dataframe
    d = {'title': titles,
         'locations': locations,
         'company': companies, 
         'company_rating': ratings,
         'post_age': ages, 
         'job_link': links, 
         'job_description': descriptions}
    df = pd.DataFrame(d)
    return df

In [72]:
# Test function acquire_page_indeed
page_num, df = acquire_page_indeed(url)

Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Data Scientist - United States - Indeed.com
Status Code:  200
Vaco Careers and Employment | Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL - Indeed.com
Status Code:  200
MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE - Birmingham, AL 35243 - Indeed.com
Status Code:  200
Statistical Analyst - Jefferson County, AL - Indeed.com
Status Code:  200
Data Scientist Intern - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Machine Learning/Artificial Intelligence Software Developer - Huntsville, AL 35805 - Indeed.com
Status Code:  200
Software Engineer/Data Scientist - Huntsville, AL 35802 - Indeed.com
Status Code:  200
Lead Financial Analyst - Artificial Intelligence Strategic Growth Offering (AI SGO) Finance & Investment - Birmingham, AL 35203 - Indeed.

In [73]:
# Print the page number
page_num

'1'

In [79]:
df.columns

Index(['title', 'locations', 'company', 'company_rating', 'post_age',
       'job_link', 'job_description'],
      dtype='object')

In [76]:
df.job_description[0]

"MTA, Inc. is a Woman Owned Small Business with headquarters in Huntsville, AL. We are a diversified company recognized for excellence in Engineering, Integrated Logistics, and Quality Assurance. MTA provides services to U.S. defense agencies, NASA, and the U.S. Corp of Engineers.\nMTA, Inc. has an immediate opening for the position of Data Scientist.\nJob Description:\nTo provide support to the System Readiness Directorate's Reliability, Availability, Maintainability and System Assessment (RAM-SA) Division in support of developing and implementing machine learning algorithms and tools in SQL and Python. The objective of this project is to develop, implement and support data science work in the RAM division in support of tool development research and the Predictive Maintenance National Mission Initiative (PMx NMI).\nDuties and responsibilities:\nPerforming detailed and complex calculations necessary to assess advanced systems concepts\nTransferring data into a new format to make it mor

In [100]:
def jobs_indeed(job_title, location):
    '''
    This function accepts the job title and location and return 
    the job information pull from Indeed.com.
    '''
    # Generate the urls based on job title and location (state)
    url = first_page_url = first_page_url_indeed(job_title, location)
    # Set up an counter
    counter = 1
    # Create an empty dataframe to hold the job information
    df_jobs = pd.DataFrame(columns = ['title', 'locations', 'company', 'company_rating', 
                                      'post_age','job_link', 'job_description'])
    # Pull the page number
    page_num = int(page_num_indeed(url))
    # Set up an checker
    keep_going = (counter == page_num)   
    # For loop through the urls to pull job information
    while keep_going and page_num <=40:
        df = acquire_page_indeed(url)
        print("--------------------------------")
        print("Page: ", page_num)
        print("--------------------------------")
        df_jobs = df_jobs.append(df, ignore_index=True)
        time.sleep(180)
        dic = {'start': page_num*10}
        relative_url = urllib.parse.urlencode(dic)
        url = first_page_url + '&' + relative_url
        counter = counter + 1
        page_num = int(page_num_indeed(url))
        keep_going = (counter == page_num)
    # Print the total number of jobs
    print(f"Total number of {job_title} positions in {location}: ", df_jobs.shape[0])
    return df_jobs

In [101]:
# Test function jobs_in_state_indeed
df = jobs_indeed('data scientist', 'al')

Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com
Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Data Scientist - United States - Indeed.com
Status Code:  200
Vaco Careers and Employment | Indeed.com
Status Code:  200
MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE - Birmingham, AL 35243 - Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL - Indeed.com
Status Code:  200
Statistical Analyst - Jefferson County, AL - Indeed.com
Status Code:  200
Data Scientist Intern - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Machine Learning/Artificial Intelligence Software Developer - Huntsville, AL 35805 - Indeed.com
Status Code:  200
Software Engineer/Data Scientist - Huntsville, AL 35802 - Indeed.com
Status Cod

In [114]:
# Print a concise summary of the df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            42 non-null     object
 1   locations        42 non-null     object
 2   company          42 non-null     object
 3   company_rating   42 non-null     object
 4   post_age         42 non-null     object
 5   job_link         42 non-null     object
 6   job_description  42 non-null     object
dtypes: object(7)
memory usage: 2.4+ KB


In [115]:
df.sample(5)

,title,locations,company,company_rating,post_age,job_link,job_description
17,Member of Technical Staff- Technical Support -...,"Huntsville, AL",Wind River,3.7,30+ days ago,https://www.indeed.com/rc/clk?jk=c768446d7ff0b...,Member of Technical Staff - Technical Support ...
16,Machine Learning Engineer,"Huntsville, AL 35806",CFD Research Corporation,4.2,28 days ago,https://www.indeed.com/rc/clk?jk=189d948471f4a...,This position will join an R&D team focused on...
3,"MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE\nnew","Birmingham, AL 35243","B.A.S.S., LLC",missing,Today,"https://www.indeed.com/company/B.A.S.S.,-LLC/j...","MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCEBasic..."
30,AI/ML Software Engineer,"Huntsville, AL",COLSA,3.9,30+ days ago,https://www.indeed.com/rc/clk?jk=fad792fe1641a...,"General Summary\n\n\nDesigns, develops, troubl..."
41,DATA SCIENCE CONSULTANT BIRMINGHAM,"Birmingham, AL 35203 (Central City area)",managementsolutions,3.8,30+ days ago,https://www.indeed.com/rc/clk?jk=aa00dcc168e47...,United States\n\nDATA SCIENCE CONSULTANT BIRMI...


In [112]:
df.job_link[41]

'https://www.indeed.com/rc/clk?jk=aa00dcc168e475b5&fccid=f3eca80b6759548b&vjs=3'

In [113]:
df.job_description[41]

'United States\n\nDATA SCIENCE CONSULTANT BIRMINGHAM\n\nBirmingham / Internship / Number of vacancies: 2\n\n\n\n\nYou will be working in key projects for leading organizations in data mining & knowledge Discovery, predictive modeling, trend modeling, Simulation models (Monte Carlo), Review of credit rating and scoring models and quant support to the business and R&D projects.\n\nRequirements\n\nRecent graduates or final year students from disciplines relating to Mathematics, Physics, Statistics, Econometrics or other Quantitative fields.\nPostgraduate studies and/or specialised courses are an asset, especially in Data Science, Quantitative Finance or similar.\nShould desirably have knowledge of modeling techniques (logit, GLM, time series, decision trees, random forests, clustering), statistical programming languages (SAS, R, Python, Matlab) and big data tools and platforms (Hadoop, Hive, etc.).\nSolid academic record.\nStrong computer skills.\nKnowledge of other languages is desirable

### Combine Job Search Results

In [126]:
# Load data scientist job posts in TX

# Import the file path
database = env_Shi.database

# Read the data scientist jobs in TX
df_ds_tx1 = pd.read_csv(f"{database}data_scientist_tx_indeed_012121.csv", index_col=0)
df_ds_tx2 = pd.read_csv(f"{database}data_scientist_tx_indeed_012221.csv", index_col=0)

In [127]:
# Print out the information
df_ds_tx1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482 entries, 0 to 481
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            482 non-null    object
 1   locations        482 non-null    object
 2   company          482 non-null    object
 3   company_rating   482 non-null    object
 4   post_age         482 non-null    object
 5   job_link         482 non-null    object
 6   job_description  482 non-null    object
dtypes: object(7)
memory usage: 30.1+ KB


In [128]:
# Print out the information
df_ds_tx2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510 entries, 0 to 509
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            510 non-null    object
 1   locations        510 non-null    object
 2   company          510 non-null    object
 3   company_rating   510 non-null    object
 4   post_age         510 non-null    object
 5   job_link         510 non-null    object
 6   job_description  510 non-null    object
dtypes: object(7)
memory usage: 31.9+ KB


In [129]:
df_ds_tx = pd.concat([df_ds_tx1, df_ds_tx2], ignore_index=True)
df_ds_tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            992 non-null    object
 1   locations        992 non-null    object
 2   company          992 non-null    object
 3   company_rating   992 non-null    object
 4   post_age         992 non-null    object
 5   job_link         992 non-null    object
 6   job_description  992 non-null    object
dtypes: object(7)
memory usage: 54.4+ KB


In [130]:
df_ds_tx.head(2)

,title,locations,company,company_rating,post_age,job_link,job_description
0,Data Scientist Associate Sr\nnew,"Plano, TX","JPMorgan Chase Bank, N.A.",3.9,Just posted,https://www.indeed.com/rc/clk?jk=fdb25f52f6062...,J.P. Morgan's Corporate & Investment Bank (CIB...
1,Senior Data Scientist\nnew,"Austin, TX 78701 (Downtown area)",Sam's Club,3.5,Just posted,https://www.indeed.com/rc/clk?jk=3d235cc44e5cc...,What you'll do...\nPosition: Senior Data Scien...


In [131]:
# Compute the number of duplicate rows 
# using columns: title, locations, company, job links and job description

df_ds_tx.duplicated(subset=['title', 'locations', 'company', 'job_link', 'job_description']).sum()

436

In [132]:
# Compute the number of new jobs posted

num_duplicateds = df_ds_tx.duplicated(subset=['title', 'locations', 'company', 
                                              'job_link', 'job_description'], keep=False).sum()
num_new_jobs = df_ds_tx.shape[0] - num_duplicateds
num_new_jobs

237

## Data Preparation

In [133]:
# Define a function to remove the duplicates

def remove_duplicates(df):
    '''
    This function removes the duplicates in the dataframe
    '''
    # Define the columns for identifying duplicates
    columns = ['title', 'locations', 'company', 'job_link', 'job_description']
    # Drop the duplicates except for the last occurrence
    df.drop_duplicates(subset=columns, keep='last', inplace=True, ignore_index=True)
    return df

In [134]:
# Test the function

df_ds_tx = remove_duplicates(df_ds_tx)
df_ds_tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            556 non-null    object
 1   locations        556 non-null    object
 2   company          556 non-null    object
 3   company_rating   556 non-null    object
 4   post_age         556 non-null    object
 5   job_link         556 non-null    object
 6   job_description  556 non-null    object
dtypes: object(7)
memory usage: 30.5+ KB


In [135]:
# Rank the companies based on the number of posts
df_ds_tx.company.value_counts()

Deloitte                          21
Apple                             15
Cognizant Technology Solutions    12
USAA                              11
Dell Technologies                  8
                                  ..
ePlus inc.                         1
Syncroness                         1
Plasma Computing Group             1
Aramco Services Company            1
CFoundations                       1
Name: company, Length: 332, dtype: int64

In [136]:
# Clean the text in the `job_description` column
df_ds_tx_clean = MVP_Bojado.prep_job_description_data(df_ds_tx, 'job_description')
df_ds_tx.head(2)

,title,locations,company,company_rating,post_age,job_link,job_description,clean,tokenized,stemmed,lemmatized
0,Data Scientist Associate Sr\nnew,"Plano, TX","JPMorgan Chase Bank, N.A.",3.9,Just posted,https://www.indeed.com/rc/clk?jk=fdb25f52f6062...,J.P. Morgan's Corporate & Investment Bank (CIB...,jp morgan corporate investment bank cib global...,jp morgans corporate investment bank cib is a ...,jp morgan corpor invest bank cib is a global l...,jp morgan corporate investment bank cib is a g...
1,Director of Data Science\nnew,"Austin, TX 78701 (Downtown area)",CyberCoders,3.3,Just posted,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Director of Data Science\nHumanity is sufferin...,director data science humanity suffering healt...,director of data science\nhumanity is sufferin...,director of data scienc human is suffer from a...,director of data science humanity is suffering...


In [137]:
# Create 'words' variable
words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df_ds_tx.clean]

# Add 'words' column to dataframe
# Column will contain lists of separated words in each repo
df_ds_tx = pd.concat([df_ds_tx, pd.DataFrame({'words': words})], axis=1)

df_ds_tx.head(2)

,title,locations,company,company_rating,post_age,job_link,job_description,clean,tokenized,stemmed,lemmatized,words
0,Data Scientist Associate Sr\nnew,"Plano, TX","JPMorgan Chase Bank, N.A.",3.9,Just posted,https://www.indeed.com/rc/clk?jk=fdb25f52f6062...,J.P. Morgan's Corporate & Investment Bank (CIB...,jp morgan corporate investment bank cib global...,jp morgans corporate investment bank cib is a ...,jp morgan corpor invest bank cib is a global l...,jp morgan corporate investment bank cib is a g...,"[jp, morgan, corporate, investment, bank, cib,..."
1,Director of Data Science\nnew,"Austin, TX 78701 (Downtown area)",CyberCoders,3.3,Just posted,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Director of Data Science\nHumanity is sufferin...,director data science humanity suffering healt...,director of data science\nhumanity is sufferin...,director of data scienc human is suffer from a...,director of data science humanity is suffering...,"[director, data, science, humanity, suffering,..."


In [14]:
# Define the function to create the words that appear in the job descriptions

def words_variables(df, companies):
    '''
    This function accepts the dataframe containing cleaned job description and 
    a list of company names and return a dictionary in which the values are the words 
    that appear in the job description. 
    '''
    # Create the words that appear all the job descritipons
    all_words = ' '.join(df.clean)
    # Create a dictionary to hold the variable all_words
    d_words = {'all': all_words}
    # For loop the companies and create the words that appear in their job descriptions
    for company in companies:
        mask = (df.company == company)
        s_company = df[mask].clean
        words = ' '.join(s_company)
        d_words[company] = words
    return d_words

In [78]:
# Apply the helper function: words_variables

companies = ['Apple', 'Deloitte', 'USAA']
dic = words_variables(df_ds_tx, companies)

dic['USAA'][:400]

'purpose job currently seeking talented data scientist 100 remote available san antonio home office us advanced technique integrate traditional nontraditional datasets method enable analytical solution applies predictive analytics machine learning simulation optimization technique generate management insight enable customerfacing application participates building analytical solution leveraging inte'

In [79]:
dic['Apple'][:400]

'summary posted jan 8 2021 weekly hour 40 role number200213058 imagine could apple new idea way becoming outstanding product service customer experience quickly bring passion dedication job there telling could accomplish apple strategic data solution sd team looking talented individual passionate crafting implementing operating analytical solution direct measurable impact apple customer sd data sci'

In [80]:
# Read the keys in the dictionary
dic.keys()

dict_keys(['all', 'Apple', 'Deloitte', 'USAA'])

In [19]:
# Define a function to compute the word frequency in the job description

def word_frequency(d_words):
    '''
    This function accept the dictionary created by function words_variables
    and return the word frequency in the job description. 
    '''
    # Read the company names from the dictionary
    companies = d_words.keys()
    # Create a dataframe to hold the word frequency
    word_counts = pd.DataFrame()
    # For loop through the companies and generate the word frequency in their job descriptions
    for company in companies:
        freq = pd.Series(d_words[company].split()).value_counts()
        word_counts = pd.concat([word_counts, freq], axis=1, sort=True)
    word_counts.columns = companies
    word_counts = word_counts.fillna(0).apply(lambda s: s.astype(int))
    word_counts.sort_values(by='all', ascending=False, inplace=True)
    return word_counts

In [81]:
# Test the function word_frequency

df_word_frequency = word_frequency(dic)
df_word_frequency.head(5)

,all,Apple,Deloitte,USAA
data,5205,133,157,91
experience,3200,91,232,105
business,2035,66,147,125
team,1925,79,100,19
work,1592,30,106,80


In [82]:
df_word_frequency.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11932 entries, data to auditoryvisual
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   all       11932 non-null  int64
 1   Apple     11932 non-null  int64
 2   Deloitte  11932 non-null  int64
 3   USAA      11932 non-null  int64
dtypes: int64(4)
memory usage: 466.1+ KB


In [86]:
mask = (df_word_frequency.index == 'python')
df_word_frequency[mask].sort_values(by='all', ascending=False).head(10)

,all,Apple,Deloitte,USAA
python,467,13,20,12


In [87]:
mask = (df_word_frequency.index == 'aws')
df_word_frequency[mask].sort_values(by='all', ascending=False).head(10)

,all,Apple,Deloitte,USAA
aws,207,0,18,3


In [88]:
mask = (df_word_frequency.index == 'sql')
df_word_frequency[mask].sort_values(by='all', ascending=False).head(10)

,all,Apple,Deloitte,USAA
sql,334,11,22,10


In [89]:
# Added 'Bigram' column to dataframe
df_ds_tx['bigrams'] = [list(nltk.ngrams(wordlist, 2)) for wordlist in df_ds_tx.words]
df_ds_tx.head()

,title,locations,company,company_rating,post_age,job_link,job_description,clean,tokenized,stemmed,lemmatized,words,bigrams
0,Data Scientist Associate Sr\nnew,"Plano, TX","JPMorgan Chase Bank, N.A.",3.9,Just posted,https://www.indeed.com/rc/clk?jk=fdb25f52f6062...,J.P. Morgan's Corporate & Investment Bank (CIB...,jp morgan corporate investment bank cib global...,jp morgans corporate investment bank cib is a ...,jp morgan corpor invest bank cib is a global l...,jp morgan corporate investment bank cib is a g...,"[jp, morgan, corporate, investment, bank, cib,...","[(jp, morgan), (morgan, corporate), (corporate..."
1,Director of Data Science\nnew,"Austin, TX 78701 (Downtown area)",CyberCoders,3.3,Just posted,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Director of Data Science\nHumanity is sufferin...,director data science humanity suffering healt...,director of data science\nhumanity is sufferin...,director of data scienc human is suffer from a...,director of data science humanity is suffering...,"[director, data, science, humanity, suffering,...","[(director, data), (data, science), (science, ..."
2,Sr Big Data/Data Engineer\nnew,"Houston, TX 77002 (Downtown area)",CFoundations,missing,Today,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,BIG DATA ENGINEERPay Rate - $80-105 per hour C...,big data engineerpay rate 80105 per hour c2c 1...,big data engineerpay rate 80105 per hour c2c o...,big data engineerpay rate 80105 per hour c2c o...,big data engineerpay rate 80105 per hour c2c o...,"[big, data, engineerpay, rate, 80105, per, hou...","[(big, data), (data, engineerpay), (engineerpa..."
3,Sr Research Associate (Data Analyst/Statistica...,"Fort Worth, TX 76107 (Arlington Heights area)",University of North Texas Health Science Center,4.1,Today,https://www.indeed.com/rc/clk?jk=55d6b96838357...,Welcome to the University of North Texas Syste...,welcome university north texas system unt worl...,welcome to the university of north texas syste...,welcom to the univers of north texa system or ...,welcome to the university of north texas syste...,"[welcome, university, north, texas, system, un...","[(welcome, university), (university, north), (..."
4,"Analyst I, Statistical (226 Days)\nnew","Dallas, TX",Dallas Independent School District,3.7,Today,https://www.indeed.com/rc/clk?jk=9ec38e7c6c285...,"Analyst I, Statistical (226 Days) -(RTP2020121...",analyst statistical 226 day rtp20201216030 des...,analyst i statistical 226 days rtp20201216030\...,analyst i statist 226 day rtp20201216030 descr...,analyst i statistical 226 day rtp20201216030 d...,"[analyst, statistical, 226, day, rtp2020121603...","[(analyst, statistical), (statistical, 226), (..."


In [22]:
# Define a function to compute the bigrams frequency in the job description

def bigrams_frequency(d_words):
    '''
    This function accept the dictionary created by function words_variables
    and return the bigrams frequency in the job description. 
    '''
    # Read the company names from the dictionary
    companies = d_words.keys()
    # Create a dataframe to hold the word frequency
    bigrams_counts = pd.DataFrame()
    # For loop through the companies and generate the word frequency in their job descriptions
    for company in companies:
        freq = pd.Series(list(nltk.ngrams(d_words[company].split(), 2))).value_counts()
        bigrams_counts = pd.concat([bigrams_counts, freq], axis=1, sort=True)
    bigrams_counts.columns = companies
    bigrams_counts = bigrams_counts.fillna(0).apply(lambda s: s.astype(int))
    bigrams_counts.sort_values(by='all', ascending=False, inplace=True)
    return bigrams_counts

In [97]:
# Compute bigrams_frequency

bigrams = bigrams_frequency(dic)
bigrams.head()

,,all,Apple,Deloitte,USAA
machine,learning,885,23,13,16
data,science,661,33,12,3
year,experience,443,5,28,26
data,scientist,416,21,5,3
computer,science,369,9,9,5


In [26]:
# Define a function to compute the trigrams frequency in the job description

def trigrams_frequency(d_words):
    '''
    This function accept the dictionary created by function words_variables
    and return the trigrams frequency in the job description. 
    '''
    # Read the company names from the dictionary
    companies = d_words.keys()
    # Create a dataframe to hold the word frequency
    trigrams_counts = pd.DataFrame()
    # For loop through the companies and generate the word frequency in their job descriptions
    for company in companies:
        freq = pd.Series(list(nltk.ngrams(d_words[company].split(), 3))).value_counts()
        trigrams_counts = pd.concat([trigrams_counts, freq], axis=1, sort=True)
    trigrams_counts.columns = companies
    trigrams_counts = trigrams_counts.fillna(0).apply(lambda s: s.astype(int))
    trigrams_counts.sort_values(by='all', ascending=False, inplace=True)
    return trigrams_counts

In [91]:
# Compute trigrams_frequency

trigrams = trigrams_frequency(dic)
trigrams.head()

,,,all,Apple,Deloitte,USAA
sexual,orientation,gender,151,0,0,0
equal,opportunity,employer,151,1,0,0
race,color,religion,143,0,0,0
orientation,gender,identity,137,0,0,0
without,regard,race,110,0,0,0


In [142]:
# Create the masks for different skills

mask_python = df_ds_tx.clean.str.contains('python')
mask_sql = df_ds_tx.clean.str.contains('sql')
mask_ml = df_ds_tx.clean.str.contains('machine learning')
mask_tableau = df_ds_tx.clean.str.contains('tableau')
mask_aws = df_ds_tx.clean.str.contains('aws')

mask = mask_python & mask_sql & mask_tableau

In [143]:
# How many companies need all three skills: python, sql and tableau
mask.sum()

67

In [149]:
df_ds_tx[mask].head(5)

,title,locations,company,company_rating,post_age,job_link,job_description,clean,tokenized,stemmed,lemmatized,words
0,Data Scientist Associate Sr\nnew,"Plano, TX","JPMorgan Chase Bank, N.A.",3.9,Just posted,https://www.indeed.com/rc/clk?jk=fdb25f52f6062...,J.P. Morgan's Corporate & Investment Bank (CIB...,jp morgan corporate investment bank cib global...,jp morgans corporate investment bank cib is a ...,jp morgan corpor invest bank cib is a global l...,jp morgan corporate investment bank cib is a g...,"[jp, morgan, corporate, investment, bank, cib,..."
7,Sr. Machine Learning R&D Engineer\nnew,"Austin, TX 78753 (Windsor Hills area)",Ultra Global Business Services,missing,Today,https://www.indeed.com/rc/clk?jk=dd698164b1c52...,Job Description:\nUltra Labs is seeking a tale...,job description ultra lab seeking talented hig...,job description\nultra labs is seeking a talen...,job descript ultra lab is seek a talent highli...,job description ultra lab is seeking a talente...,"[job, description, ultra, lab, seeking, talent..."
9,Senior Data Scientist\nnew,"Austin, TX 78727",Cvent,3.6,Today,https://www.indeed.com/rc/clk?jk=0fb6ee6e19aba...,The mission of the Cvent data science team is ...,mission cvent data science team enhance cvents...,the mission of the cvent data science team is ...,the mission of the cvent data scienc team is t...,the mission of the cvent data science team is ...,"[mission, cvent, data, science, team, enhance,..."
15,Single Family - Data Science - Associate\nnew,"Plano, TX",Fannie Mae,4.0,3 days ago,https://www.indeed.com/rc/clk?jk=ab7209945854d...,": At Fannie Mae, futures are made. The inspiri...",fannie mae future made inspiring work make aff...,at fannie mae futures are made the inspiring w...,at fanni mae futur are made the inspir work we...,at fannie mae future are made the inspiring wo...,"[fannie, mae, future, made, inspiring, work, m..."
16,Commercial Banking - Senior Data Scientist - V...,"Plano, TX","JPMorgan Chase Bank, N.A.",3.9,4 days ago,https://www.indeed.com/rc/clk?jk=97c7b07d683f0...,The Corporate Client Banking & Specialized Ind...,corporate client banking specialized industry ...,the corporate client banking specialized indus...,the corpor client bank special industri ccbsi ...,the corporate client banking specialized indus...,"[corporate, client, banking, specialized, indu..."


### LinkedIn API

In [21]:
access_token = env.access_token
url = "https://api.linkedin.com/v2/recommendedJobs?Bearer"

In [13]:
headers = {'Content-type': 'application/json',
           'X-Restli-Protocol-Version': '2.0.0',
           'Authorization': 'Bearer' + access_token}

In [14]:
response = requests.get(url, headers=headers)
response.ok

False

In [15]:
response.status_code

401